# 빅데이터 분석기사 실기 준비 놀이터
- https://www.kaggle.com/datasets/agileteam/bigdatacertificationkr

## T2-4. House Prices (Regression) / 집값 예측 / RMSE

### 2. 집값 예측
- 데이터셋 : House Prices - Advanced Regression Techniques_train.csv
- 이상치 처리 : Glucose, BloodPressure, SkinThickness, Insulin, BMI가 0인 값
- https://www.kaggle.com/code/agileteam/house-prices-starter-xgb

#### 0. 시험 환경 세팅

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./data/House Prices - Advanced Regression Techniques_train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='SalePrice', id_name='Id')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1168, 80), (292, 80), (1168, 2), (292, 2))

#### 1. 라이브러리 및 데이터 호출

In [2]:
import pandas as pd
import numpy as np

X_train.shape, X_test.shape, y_train.shape

((1168, 80), (292, 80), (1168, 2))

In [3]:
X_train.drop('Id', axis=1, inplace=True)
Id = X_test.pop('Id')

y = y_train['SalePrice']

#### 2. EDA

In [4]:
display(X_train.info(), X_train.head(), X_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 81 to 1140
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1168 non-null   int64  
 1   MSZoning       1168 non-null   object 
 2   LotFrontage    956 non-null    float64
 3   LotArea        1168 non-null   int64  
 4   Street         1168 non-null   object 
 5   Alley          70 non-null     object 
 6   LotShape       1168 non-null   object 
 7   LandContour    1168 non-null   object 
 8   Utilities      1168 non-null   object 
 9   LotConfig      1168 non-null   object 
 10  LandSlope      1168 non-null   object 
 11  Neighborhood   1168 non-null   object 
 12  Condition1     1168 non-null   object 
 13  Condition2     1168 non-null   object 
 14  BldgType       1168 non-null   object 
 15  HouseStyle     1168 non-null   object 
 16  OverallQual    1168 non-null   int64  
 17  OverallCond    1168 non-null   int64  
 18  YearBui

None

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
81,120,RM,32.0,4500,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,3,2006,WD,Normal
1418,20,RL,71.0,9204,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2008,COD,Normal
1212,30,RL,50.0,9340,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2009,WD,Normal
588,20,RL,65.0,25095,Pave,NaN,IR1,Low,AllPub,Inside,...,60,0,NaN,NaN,NaN,0,6,2009,WD,Partial
251,120,RM,44.0,4750,Pave,NaN,IR1,HLS,AllPub,Inside,...,153,0,NaN,NaN,NaN,0,12,2007,WD,Family


MSSubClass         0
MSZoning           0
LotFrontage      212
LotArea            0
Street             0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 79, dtype: int64

In [5]:
display(X_test.info(), X_test.head(), X_test.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292 entries, 1380 to 906
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     292 non-null    int64  
 1   MSZoning       292 non-null    object 
 2   LotFrontage    245 non-null    float64
 3   LotArea        292 non-null    int64  
 4   Street         292 non-null    object 
 5   Alley          21 non-null     object 
 6   LotShape       292 non-null    object 
 7   LandContour    292 non-null    object 
 8   Utilities      292 non-null    object 
 9   LotConfig      292 non-null    object 
 10  LandSlope      292 non-null    object 
 11  Neighborhood   292 non-null    object 
 12  Condition1     292 non-null    object 
 13  Condition2     292 non-null    object 
 14  BldgType       292 non-null    object 
 15  HouseStyle     292 non-null    object 
 16  OverallQual    292 non-null    int64  
 17  OverallCond    292 non-null    int64  
 18  YearBui

None

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1380,30,RL,45.0,8212,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
520,190,RL,60.0,10800,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal
1175,50,RL,85.0,10678,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2007,WD,Normal
351,120,RL,NaN,5271,Pave,NaN,IR1,Low,AllPub,Inside,...,184,0,NaN,NaN,NaN,0,12,2006,WD,Abnorml
1335,20,RL,80.0,9650,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2009,WD,Normal


MSSubClass        0
MSZoning          0
LotFrontage      47
LotArea           0
Street            0
                 ..
MiscVal           0
MoSold            0
YrSold            0
SaleType          0
SaleCondition     0
Length: 79, dtype: int64

#### 3. Preprocessing
- 컬럼 중 수치형 컬럼만 선택
- 결측치 컬럼 : 평균 대체
- 스케일링 : 정규화

> 컬럼 중 수치형 컬럼만 선택

In [6]:
int_col = X_train.select_dtypes(exclude='object').columns
X_train = X_train[int_col]
X_test = X_test[int_col]

X_train.shape, X_test.shape

((1168, 36), (292, 36))

> 결측치 처리 : 평균 대체

In [7]:
isnull_col = X_train.isnull().sum()
isnull_col = list(isnull_col[isnull_col>0].index)

print(isnull_col)

isnull_col = X_test.isnull().sum()
isnull_col = list(isnull_col[isnull_col>0].index)

print(isnull_col)

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
['LotFrontage', 'MasVnrArea', 'GarageYrBlt']


In [8]:
for c in isnull_col:
    print(c, X_train[c].nunique())
    print('train mean :', X_train[c].mean())
    print('train median :', X_train[c].median())
    print('train mode :', X_train[c].mode()[0])
    print('='*20)
    print('test mean :', X_test[c].mean())
    print('test median :', X_test[c].median())
    print('test mode :', X_test[c].mode()[0])
    print('='*100)

LotFrontage 105
train mean : 70.07008368200837
train median : 69.0
train mode : 60.0
test mean : 69.97142857142858
test median : 70.0
test mode : 60.0
MasVnrArea 286
train mean : 106.42340791738383
train median : 0.0
train mode : 0.0
test mean : 92.71379310344828
test median : 0.0
test mode : 0.0
GarageYrBlt 94
train mean : 1978.684733514002
train median : 1980.0
train mode : 2005.0
test mean : 1977.7794117647059
test median : 1978.0
test mode : 2006.0


In [9]:
for c in isnull_col:
    X_train[c] = X_train[c].fillna(X_train[c].mean(), axis=0)
    X_test[c] = X_test[c].fillna(X_test[c].mean(), axis=0)

In [10]:
X_train.isnull().sum().sum(), X_test.isnull().sum().sum()

(0, 0)

> 스케일링 : 정규화

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train.shape, X_test.shape

((1168, 36), (292, 36))

#### 4. Modeling

In [12]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor, XGBRFRegressor

from sklearn.metrics import mean_squared_error, r2_score

In [13]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size=0.2, shuffle=True, random_state=156)

In [14]:
rf = RandomForestRegressor(random_state=156)
gb = GradientBoostingRegressor(random_state=156)
xgb = XGBRegressor(random_state=156)
xgbrf = XGBRFRegressor(random_state=156)

models = [rf, gb, xgb, xgbrf]

In [15]:
for model in models:
    model.fit(X_tr, y_tr)
    pred = model.predict(X_val)
    
    name = str(model).split('(')[0]
    print(name)
    print(np.sqrt(mean_squared_error(y_val, pred)))
    print(r2_score(y_val, pred))
    print('='*100)

RandomForestRegressor
29320.96947767194
0.8836366153218345
GradientBoostingRegressor
26903.542390179704
0.9020332663935836
XGBRegressor
36228.51861352739
0.8223517860260546
XGBRFRegressor
32995.0184307724
0.8526478611830118


In [16]:
gb = GradientBoostingRegressor(random_state=156)

gb.fit(X_train, y)
pred = gb.predict(X_test)

In [17]:
out_data = pd.DataFrame({'Id':Id, 'SalePrice':pred})
out_data

,Id,SalePrice
1380,1381,73574.356104
520,521,97954.238811
1175,1176,327271.110119
351,352,202855.654640
1335,1336,157411.072985
...,...,...
1283,1284,140790.918758
1039,1040,82626.992727
61,62,94909.652624
1395,1396,319968.951627


In [18]:
# out_data.to_csv('123456,csv', index=False)

In [19]:
# 가채점
print(np.sqrt(mean_squared_error(y_test.SalePrice, pred)))
print(r2_score(y_test.SalePrice, pred))

23755.871417687908
0.9032917104221649


#### 5. 다른 모델 평가

In [20]:
for model in models:
    model.fit(X_train, y)
    pred = model.predict(X_test)
    
    name = str(model).split('(')[0]
    print(name)
    print('RMSE :', np.sqrt(mean_squared_error(y_test.SalePrice, pred)))
    print('r2 :', r2_score(y_test.SalePrice, pred))
    print('='*100)

RandomForestRegressor
RMSE : 26054.170084320638
r2 : 0.8836741436680863
GradientBoostingRegressor
RMSE : 23755.871417687908
r2 : 0.9032917104221649
XGBRegressor
RMSE : 25323.086648321376
r2 : 0.8901107896923101
XGBRFRegressor
RMSE : 27568.727965740425
r2 : 0.8697567495022234
